In [12]:
# https://stackoverflow.com/questions/16515099/saving-a-stream-while-playing-it-using-libvlc
# vlc your_input_file_or_stream_here --sout="#std{access=file,mux=ps,dst=go.mpg}" 
import vlc
import os
import time
import numpy as np
import requests

from IPython.display import clear_output


In [13]:
#  0: 'NothingSpecial',
#  1: 'Opening',
#  2: 'Buffering',
#  3: 'Playing',
#  4: 'Paused',
#  5: 'Stopped',
#  6: 'Ended',
#  7: 'Error'

In [85]:
import vlc
import time
import numpy as np
import multiprocessing

class ExperimentClient():
    def __init__(self, url='http://vimdemo1.cs.upb.de:9000/data/stream_1.m3u8'):
        stream_url = url
        instance = vlc.Instance("--vout=dummy --aout=dummy".split())
        
        self.media = instance.media_new(stream_url)
        self.player = instance.media_player_new()

        self.player.set_media(self.media)

        self.bitrates = []
        self.playing_time = 0

    def restart_player(self):
        self.player.stop()
        self.player.play()

    def start_player(self):
        self.player.play()

    def stop_player(self):
        self.player.stop()

    def consume_stream_proc(self, timeblock, return_dict):
        _counter = 0
        # return_dict = {
        #     "bitrates": [],
        #     "playing_time": 0
        # }
        _just_starting = True
        self.start_player()
        # init_time = time.time()
        previous_bitrate = 0.0
        _same_bitrate_count = 0
        SAME_BITRATE_THRESHOLD = 2
        time.sleep(1)
        while(_counter < timeblock+1):
            try:
                stats = vlc.MediaStats() 
                state = str(self.media.get_state())
                _bitrate = 0.0
                if state in ["State.Opening", "State.Buffering", "State.Playing"]:
                    self.media.get_stats(stats)
                    _bitrate = stats.demux_bitrate*8000.0

                    print("Bitrate")
                    print(_bitrate)
                    print(state)
                    if _bitrate == 0.0:
                        if _just_starting:
                            print("Just starting!")
                            _just_starting = False
                        else:
                            print("No data yet...")
                            return_dict['down_time'] += 1
                    else:
                        if _bitrate == previous_bitrate:
                            if _same_bitrate_count >= SAME_BITRATE_THRESHOLD:
                                _same_bitrate_count = 0
                                print("Same bitrate, Retrying...")
                                return_dict['down_time'] += SAME_BITRATE_THRESHOLD + 1
                                return_dict['playing_time'] -= SAME_BITRATE_THRESHOLD
                                self.restart_player()
                            else:
                                print("Same bitrate ++ !!")
                                _same_bitrate_count += 1
                                return_dict['bitrates'].append(_bitrate)
                                return_dict['playing_time'] += 1
                        else:
                            _same_bitrate_count = 0
                            return_dict['bitrates'].append(_bitrate)
                            return_dict['playing_time'] += 1

                else:
                    if _just_starting:
                        print("Just starting!")
                        _just_starting = False                    
                    else:
                        print("Retrying...")
                        return_dict['down_time'] += 1                    
                        self.restart_player()

            except Exception as e:
                print(e)

            print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
            print("\n\n")
            previous_bitrate = _bitrate
            time.sleep(1)
            _counter += 1

        self.stop_player()        
        return return_dict


In [86]:
MANO_URL = "thesismano1.cs.upb.de"
def set_remote_version(version, host=MANO_URL, port=8898) :
    if "virtual_deployment_units_vm" in version:
        switch_type="VM"
    if "virtual_deployment_units_gpu" in version:
        switch_type="GPU"
    if "virtual_deployment_units_con" in version:
        switch_type="CON"



    _base_path = 'http://{0}:{1}/switch_version?version={2}'.format(host, port, switch_type)

    try:
        r = requests.get(_base_path, verify=False)
        print("Switch Version")
        print(r.text)
    except Exception as e:
        print(e)
        print("Switch version could'nt be set")


In [87]:
from IPython.display import clear_output
# clear_output(wait=True)

manager = multiprocessing.Manager()
return_dict = manager.dict()

return_dict["bitrates"] = manager.list()
return_dict["playing_time"] = 0
return_dict["down_time"] = 0

_runner = ExperimentClient()

# output = _runner.consume_stream_proc(60, {})
timeblock = 60
p = multiprocessing.Process(target=_runner.consume_stream_proc, args=(timeblock, return_dict))
p.start()
p_killed = False

while(p.is_alive()):
    # print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
    # print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
    # clear_output(wait=True)
    time.sleep(1)

print("\n####################")
print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
print("####################\n")

p.terminate()
# set_remote_version("virtual_deployment_units_con:transcoder-image-1-con")
set_remote_version("virtual_deployment_units_gpu:transcoder-image-1-gpu")

return_dict = manager.dict()

return_dict["bitrates"] = manager.list()
return_dict["playing_time"] = 0
return_dict["down_time"] = 0

p = multiprocessing.Process(target=_runner.consume_stream_proc, args=(timeblock, return_dict))
p.start()

while(p.is_alive()):
    # print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
    # print(return_dict["bitrates"][-5:], return_dict["playing_time"], return_dict["down_time"])
    # clear_output(wait=True)
    time.sleep(1)

print("\n####################")
print(np.mean(return_dict["bitrates"]), return_dict["playing_time"], return_dict["down_time"])
print("####################\n")

#     if req_serv_id in self.active_services:
#         # LOG.info("Still active..." + req_serv_id)
#         pass
#     else:
#         LOG.info("Killing training..." + req_serv_id)
#         p.terminate()
#         p_killed = True




Bitrate
0.0
State.Playing
Just starting!
[] 0 0



Bitrate
934.8301887512207
State.Playing
[934.8301887512207] 1 0



Bitrate
727.5535464286804
State.Playing
[934.8301887512207, 727.5535464286804] 2 0



Bitrate
832.5671553611755
State.Playing
[934.8301887512207, 727.5535464286804, 832.5671553611755] 3 0



Bitrate
667.3272252082825
State.Playing
[934.8301887512207, 727.5535464286804, 832.5671553611755, 667.3272252082825] 4 0



Bitrate
667.3272252082825
State.Playing
Same bitrate ++ !!
[934.8301887512207, 727.5535464286804, 832.5671553611755, 667.3272252082825, 667.3272252082825] 5 0



Bitrate
1467.963457107544
State.Playing
[727.5535464286804, 832.5671553611755, 667.3272252082825, 667.3272252082825, 1467.963457107544] 6 0



Bitrate
356.18525743484497
State.Playing
[832.5671553611755, 667.3272252082825, 667.3272252082825, 1467.963457107544, 356.18525743484497] 7 0



Bitrate
1186.9075298309326
State.Playing
[667.3272252082825, 667.3272252082825, 1467.963457107544, 356.18525743484497